<a href="https://colab.research.google.com/github/sanjana-vivek/Longitudinal-voice-deterioration-in-Parkinson-s-patients-/blob/main/Novel_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upgrade basics
!pip install --upgrade pip setuptools wheel

# Core audio + processing
!pip install librosa soundfile matplotlib pandas tqdm scipy scikit-learn

# Praat bindings
!pip install praat-parselmouth

# Nonlinear & recurrence
!pip install nolds

# Wavelet libraries
!pip install PyWavelets

# Wavelet scattering
!pip install kymatio

# Transformers & torch
!pip install transformers torch

# Optional: openSMILE wrapper
# !pip install opensmile


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 58.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [kymatio]


In [ ]:

import os
import glob
import math
import json
from datetime import datetime
from pathlib import Path
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import librosa
import soundfile as sf
import pywt
import nolds
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from scipy.signal import resample, medfilt
from scipy.stats import skew, kurtosis


#imports that might be heavy/fail - wrapping in try/except for graceful fallback
try:
    import parselmouth  # praat bindings
    from parselmouth.praat import call as praat_call
    PRAAT_AVAILABLE = True
except Exception as e:
    print("parselmouth not available:", e)
    PRAAT_AVAILABLE = False

try:
    import kymatio
    from kymatio import Scattering1D
    KYMATIO_AVAILABLE = True
except Exception as e:
    print("kymatio not available:", e)
    KYMATIO_AVAILABLE = False

try:
    import torch
    from transformers import Wav2Vec2Processor, Wav2Vec2Model
    TRANSFORMERS_AVAILABLE = True
except Exception as e:
    print("transformers/torch import failed:", e)
    TRANSFORMERS_AVAILABLE = False

print("PRAAT_AVAILABLE:", PRAAT_AVAILABLE, "KYMATIO_AVAILABLE:", KYMATIO_AVAILABLE, "TRANSFORMERS_AVAILABLE:", TRANSFORMERS_AVAILABLE)


/usr/local/lib/python3.12/dist-packages/nolds/datasets.py:9: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


PRAAT_AVAILABLE: True KYMATIO_AVAILABLE: True TRANSFORMERS_AVAILABLE: True


In [ ]:
# Torch / GPU environment check
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("CUDA device count:", torch.cuda.device_count())
    print("Current device:", torch.cuda.current_device())
    print("Device name:", torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print("Running on CPU only")


Torch version: 2.8.0+cu126
CUDA available: False
Running on CPU only


In [ ]:
# Mounting Google Drive:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import glob, os


BASE_PATH = "/content/drive/My Drive/pd_data/denoised-speech-dataset"

# Look recursively for wav files (case-insensitive)
wav_files = glob.glob(os.path.join(BASE_PATH, "**", "*.wav"), recursive=True)
wav_files += glob.glob(os.path.join(BASE_PATH, "**", "*.WAV"), recursive=True)

print(f"Total audio files found: {len(wav_files)}")
print("Sample files:", wav_files[:5])


# import os
# subfolder = os.path.join(BASE_PATH, "emma")
# print("Files in emma:", os.listdir(subfolder)[:10])



Total audio files found: 578
Sample files: ['/content/drive/My Drive/pd_data/denoised-speech-dataset/LW/LW20.wav', '/content/drive/My Drive/pd_data/denoised-speech-dataset/LW/LW13.wav', '/content/drive/My Drive/pd_data/denoised-speech-dataset/LW/LW10.wav', '/content/drive/My Drive/pd_data/denoised-speech-dataset/LW/LW15.wav', '/content/drive/My Drive/pd_data/denoised-speech-dataset/LW/LW1.wav']


In [ ]:
pip install pyroomacoustics

In [ ]:
# Basic audio helpers and file discovery according to your structure
BASE_DIR = "/content/drive/MyDrive/pd_data/denoised-speech-dataset"

TARGET_SR = 16000  # wav2vec requirement and standardize

def list_all_wavs(base_dir=BASE_DIR):
    pattern = os.path.join(base_dir, "**", "*.wav")
    files = sorted(glob.glob(pattern, recursive=True))
    return files

def load_resample(path, sr=TARGET_SR):
    # read with soundfile to preserve bit depth, then resample if needed
    y, orig_sr = sf.read(path, always_2d=False)
    if y.ndim > 1:
        y = np.mean(y, axis=1)  # to mono
    if orig_sr != sr:
        y = librosa.resample(y.astype(np.float32), orig_sr, sr)
    return y.astype(np.float32), sr

def vad_trim(y, sr=TARGET_SR, top_db=25):
    # simple energy-based trim using librosa
    intervals = librosa.effects.split(y, top_db=top_db)
    if len(intervals) == 0:
        return y  # nothing to trim
    trimmed = np.concatenate([y[s:e] for s,e in intervals])
    return trimmed

# Discover files for sanity check
all_wavs = list_all_wavs()
print(f"Found {len(all_wavs)} .wav files under {BASE_DIR}. Example (first 10):")
for p in all_wavs[:10]:
    print(" ", p)
print("Cell 4 finished: helper functions ready and dataset scanned.")


Found 578 .wav files under /content/drive/MyDrive/pd_data/denoised-speech-dataset. Example (first 10):
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL1.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL10.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL16.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL17.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL18.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL19.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL2.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL21.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL22.wav
  /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL23.wav
Cell 4 finished: helper functions ready and dataset scanned.


In [ ]:
# SPL frame RMS -> dB computations and session stats
def framewise_rms_db(y, sr=TARGET_SR, frame_ms=25, hop_ms=10, amin=1e-9):
    frame_len = int(sr * frame_ms / 1000)
    hop_len = int(sr * hop_ms / 1000)
    if len(y) < frame_len:
        # pad short signals
        pad = frame_len - len(y)
        y = np.pad(y, (0, pad))
    frames = librosa.util.frame(y, frame_length=frame_len, hop_length=hop_len).astype(np.float32)
    rms = np.sqrt(np.mean(frames**2, axis=0) + amin)
    db = 20 * np.log10(rms + amin)  # relative dB (not calibrated SPL)
    return db

def session_spl_stats(y, sr=TARGET_SR):
    db = framewise_rms_db(y, sr)
    stats = {
        "spl_mean_db": float(np.mean(db)),
        "spl_peak_db": float(np.max(db)),
        "spl_std_db": float(np.std(db)),
        "spl_frames": int(len(db))
    }
    print(f"[SPL] mean={stats['spl_mean_db']:.2f} dB, peak={stats['spl_peak_db']:.2f} dB, frames={stats['spl_frames']}")
    return stats

# Demo on a sample file (if present)
if len(all_wavs) > 0:
    path = all_wavs[0]
    y, sr = load_resample(path)
    y_trim = vad_trim(y, sr)
    print("File used for demo:", path)
    spl_demo_raw = session_spl_stats(y, sr)
    spl_demo_trim = session_spl_stats(y_trim, sr)
else:
    print("No wav files found to demo SPL calculations.")
print("Cell 5 finished: SPL calculation functions available (printed demo stats).")


File used for demo: /content/drive/MyDrive/pd_data/denoised-speech-dataset/DL/DL1.wav
[SPL] mean=-53.47 dB, peak=-22.08 dB, frames=898
[SPL] mean=-36.69 dB, peak=-21.60 dB, frames=523
Cell 5 finished: SPL calculation functions available (printed demo stats).


In [ ]:
import os
import numpy as np
import librosa

# Optional: use parselmouth if available
try:
    import parselmouth
    PRAAT_AVAILABLE = True
except ImportError:
    PRAAT_AVAILABLE = False
    print("Parselmouth not available. Fallback to librosa for F0 only.")

# ------------------------------
# Helper functions (adjust as needed)
# ------------------------------
def load_resample(path, target_sr=16000):
    y, sr = librosa.load(path, sr=None)
    if sr != target_sr:
        y = librosa.resample(y, orig_sr=sr, target_sr=target_sr)
        sr = target_sr
    return y, sr

def vad_trim(y, sr):
    # Simple energy-based VAD trim
    y_trim, _ = librosa.effects.trim(y, top_db=30)
    return y_trim

# ------------------------------
# Classical feature extraction
# ------------------------------
def classical_features(path):
    y, sr = load_resample(path)
    y_trim = vad_trim(y, sr)
    feats = {}

    if PRAAT_AVAILABLE:
        try:
            snd = parselmouth.Sound(y_trim, sr)

            # --- Pitch (F0 statistics)
            try:
                pitch = snd.to_pitch()
                f0_values = pitch.selected_array['frequency']
                f0_values = f0_values[f0_values > 0]
                feats['f0_median'] = float(np.median(f0_values)) if len(f0_values) > 0 else np.nan
                feats['f0_std'] = float(np.std(f0_values)) if len(f0_values) > 0 else np.nan
            except Exception as e:
                feats['f0_median'] = np.nan
                feats['f0_std'] = np.nan
                print("Pitch extraction error:", e)

            # --- Jitter & Shimmer
            try:
                pp = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)
                feats['jitter_local'] = float(parselmouth.praat.call(pp, "Get jitter (local)", 0, 0, 75, 500, 1.3))
                feats['shimmer_local'] = float(parselmouth.praat.call(pp, "Get shimmer (local)", 0, 0, 75, 500, 1.3, 1.6, 0.03, 1.6))
            except Exception as e:
                feats['jitter_local'] = np.nan
                feats['shimmer_local'] = np.nan
                print("Jitter/Shimmer error:", e)

            # --- HNR
            try:
                ham = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1)
                feats['hnr_mean'] = float(parselmouth.praat.call(ham, "Get mean", 0, 0))
            except Exception as e:
                feats['hnr_mean'] = np.nan
                print("HNR error:", e)

            # --- Formants F1-F3
            try:
                form = snd.to_formant_burg()
                def form_mean(idx):
                    times = np.linspace(0, snd.duration, num=50)
                    vals = []
                    for t in times:
                        v = parselmouth.praat.call(form, "Get value at time", idx, t, "Hertz", "Linear")
                        if v > 0: vals.append(v)
                    return float(np.mean(vals)) if len(vals) > 0 else np.nan
                feats['f1_mean'] = form_mean(1)
                feats['f2_mean'] = form_mean(2)
                feats['f3_mean'] = form_mean(3)
            except Exception as e:
                feats['f1_mean'] = np.nan
                feats['f2_mean'] = np.nan
                feats['f3_mean'] = np.nan
                print("Formants error:", e)

        except Exception as e:
            print("Parselmouth failed for file:", path, e)
            # mark all as NaN
            for key in ['f0_median','f0_std','jitter_local','shimmer_local','hnr_mean','f1_mean','f2_mean','f3_mean']:
                feats[key] = np.nan

    else:
        # --- Fallback: librosa pitch estimation (pyin)
        try:
            f0, voiced_flag, voiced_probs = librosa.pyin(y_trim, fmin=50, fmax=500, sr=sr)
            f0_vals = f0[~np.isnan(f0)]
            feats['f0_median'] = float(np.median(f0_vals)) if len(f0_vals) > 0 else np.nan
            feats['f0_std'] = float(np.std(f0_vals)) if len(f0_vals) > 0 else np.nan
        except Exception as e:
            feats['f0_median'] = np.nan
            feats['f0_std'] = np.nan
            print("librosa pyin failed:", e)
        # Other classical features cannot be reliably computed without Praat
        for key in ['jitter_local','shimmer_local','hnr_mean','f1_mean','f2_mean','f3_mean']:
            feats[key] = np.nan
        print("Praat not available: classical features partially computed via librosa where possible.")

    print(f"[Classical] Extracted features for: {os.path.basename(path)}")
    return feats

# ------------------------------
# Demo on first WAV
# ------------------------------
if len(all_wavs) > 0:
    sample_file = all_wavs[0]
    cf = classical_features(sample_file)
    print("Sample classical features:\n", cf)
else:
    print("No WAV files found for demo.")


NameError: name 'all_wavs' is not defined

In [ ]:
# Non-linear and complexity features
from scipy.stats import entropy
import nolds

def spectral_entropy(y, sr=TARGET_SR, n_fft=2048, hop_length=512):
    S = np.abs(librosa.stft(y, n_fft=n_fft, hop_length=hop_length))**2
    ps = S / (np.sum(S, axis=0, keepdims=True) + 1e-12)
    ent = np.mean([entropy(col + 1e-12) for col in ps.T])
    return float(ent)

def permutation_entropy(y, order=3, delay=1):
    x = y.copy()
    if len(x) > 10000:
        x = librosa.resample(x, orig_sr=TARGET_SR, target_sr=1000)
    n = len(x)
    patterns = {}
    for i in range(n - (order-1)*delay):
        window = x[i:i+order*delay:delay]
        ranks = tuple(np.argsort(window))
        patterns[ranks] = patterns.get(ranks, 0) + 1
    ps = np.array(list(patterns.values()), dtype=float)
    ps = ps / (ps.sum() + 1e-12)
    return float(-np.sum(ps * np.log2(ps + 1e-12)))

def recurrence_rate(y, emb_dim=3, tau=1, eps_factor=0.1):
    N = len(y)
    M = N - (emb_dim-1)*tau
    if M <= 0:
        return np.nan
    X = np.zeros((M, emb_dim))
    for i in range(M):
        for j in range(emb_dim):
            X[i, j] = y[i + j * tau]
    X = (X - X.mean(axis=0)) / (X.std(axis=0) + 1e-9)
    dists = np.sqrt(((X[:, None, :] - X[None, :, :])**2).sum(axis=2))
    eps = eps_factor * np.median(dists)
    rr = np.sum(dists < eps) / (M*M)
    return float(rr)

def rpde_approx(y):
    # approximate predictability via short-window AR residuals (simple)
    from sklearn.linear_model import LinearRegression
    y = (y - np.mean(y)) / (np.std(y) + 1e-9)
    win = int(0.5 * TARGET_SR)
    step = int(0.25 * TARGET_SR)
    errs = []
    for s in range(0, max(1, len(y)-win), step):
        seg = y[s:s+win]
        if len(seg) < 50:
            continue
        order = 4
        if len(seg) < order + 1:
            continue
        X = np.vstack([seg[i: i+len(seg)-order] for i in range(order)]).T
        ytarget = seg[order:]
        try:
            lr = LinearRegression().fit(X, ytarget)
            pred = lr.predict(X)
            errs.append(np.mean((pred - ytarget)**2))
        except:
            pass
    return float(np.mean(errs)) if len(errs)>0 else np.nan

def nonlinear_features(path):
    y, sr = load_resample(path)
    y_trim = vad_trim(y, sr)
    feats = {
        "spectral_entropy": spectral_entropy(y_trim, sr),
        "perm_entropy": permutation_entropy(y_trim),
        "recurrence_rate": recurrence_rate(y_trim),
        "rpde_approx": rpde_approx(y_trim)
    }
    print(f"[NonLinear] computed for {os.path.basename(path)}: entropy={feats['spectral_entropy']:.3f}, perm_ent={feats['perm_entropy']:.3f}")
    return feats

# Demo
if len(all_wavs)>0:
    print(nonlinear_features(all_wavs[0]))
else:
    print("No files for non-linear demo.")
print("Processing finished: non-linear features functions ready.")


In [ ]:
# Wavelet band energies (pywt) and scattering (kymatio)
def wavelet_band_energy(y, wavelet='db4', levels=5):
    coeffs = pywt.wavedec(y, wavelet, level=levels)
    energies = [float(np.sum(c**2)) for c in coeffs]
    total = sum(energies) + 1e-12
    ratios = [e/total for e in energies]
    feats = {f"wavelet_energy_{i}": energies[i] for i in range(len(energies))}
    feats.update({f"wavelet_ratio_{i}": ratios[i] for i in range(len(ratios))})
    print(f"[Wavelet] computed {len(energies)} band energies")
    return feats

def wavelet_scattering_feats(y):
    if not KYMATIO_AVAILABLE:
        print("kymatio not available: skipping scattering features.")
        return {}
    # Kymatio requires length; pad/truncate to next power of two length for stability
    N = len(y)
    L = 1 << (N-1).bit_length()
    if L < N:
        L <<= 1
    y_pad = np.zeros(L, dtype=np.float32)
    y_pad[:N] = y
    scattering = Scattering1D(J=6, shape=L, Q=8)
    import torch
    Sx = scattering(torch.from_numpy(y_pad).float().unsqueeze(0))
    Sx_np = Sx.squeeze(0).numpy()
    feats = {}
    # pool mean/std across time dimension
    if Sx_np.ndim == 2:
        for i in range(Sx_np.shape[0]):
            feats[f"scat_mean_{i}"] = float(Sx_np[i].mean())
            feats[f"scat_std_{i}"] = float(Sx_np[i].std())
    print(f"[Scattering] produced {len(feats)//2} coefficients (mean+std)")
    return feats

# Demo
if len(all_wavs)>0:
    y, sr = load_resample(all_wavs[0])
    print(wavelet_band_energy(y))
    print(wavelet_scattering_feats(y))
else:
    print("No files for wavelet demo.")
print("Processing finished: wavelet features ready (scattering optional).")


In [ ]:
# wav2vec embeddings summary + baseline-drift
# Note: this uses HuggingFace Wav2Vec2 via transformers. Ensure transformers and torch are installed.

if TRANSFORMERS_AVAILABLE:
    processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")
    model.eval()
else:
    processor = None
    model = None

def extract_wav2vec_stats(y, sr=TARGET_SR):
    if not TRANSFORMERS_AVAILABLE:
        print("transformers not available: skipping embedding extraction.")
        return {}
    # processor expects numpy array, sampling rate TARGET_SR
    import torch
    input_values = processor(y, sampling_rate=sr, return_tensors="pt", padding=True).input_values
    with torch.no_grad():
        outputs = model(input_values)
    emb = outputs.last_hidden_state.squeeze(0).cpu().numpy()  # frames x dim
    stats = {
        "emb_mean_mean": float(np.mean(emb.mean(axis=0))),
        "emb_mean_std": float(np.std(emb.mean(axis=0))),
        "emb_flat_skew": float(skew(emb.flatten())),
        "emb_flat_kurtosis": float(kurtosis(emb.flatten())),
        "emb_frames": int(emb.shape[0])
    }
    print(f"[Embeddings] frames={stats['emb_frames']}, mean_mean={stats['emb_mean_mean']:.4f}")
    return stats

def embedding_baseline_drift(y_emb, baseline_emb):
    # y_emb, baseline_emb are arrays frames x dim (or mean vectors)
    if y_emb is None or baseline_emb is None:
        return {"embedding_drift": np.nan}
    # reduce to mean vectors if frames x dim
    y_mean = np.mean(y_emb, axis=0) if y_emb.ndim==2 else y_emb
    b_mean = np.mean(baseline_emb, axis=0) if baseline_emb.ndim==2 else baseline_emb
    # cosine distance
    cos_sim = 1 - cdist([y_mean], [b_mean], metric='cosine')[0,0]
    drift = float(1 - cos_sim)  # distance measure
    return {"embedding_drift": drift}

# Demo
if len(all_wavs)>0 and TRANSFORMERS_AVAILABLE:
    y, sr = load_resample(all_wavs[0])
    emb_stats = extract_wav2vec_stats(y, sr)
    print(emb_stats)
else:
    print("No embedding demo (either no files or transformers not available).")
print("Cell 9 finished: embedding feature functions ready (baseline-drift function included).")


In [ ]:
# Full per-patient pipeline that extracts all features, computes baseline normalization,
# per-feature slope, a simple deterioration_score and CUSUM detection; then plots deterioration line.

import matplotlib.dates as mdates
from datetime import datetime

def extract_features_for_file(path):
    """Extract all features for a single WAV file and return dict"""
    feats = {}
    y, sr = load_resample(path)
    y_trim = vad_trim(y, sr)
    # SPL
    feats.update(session_spl_stats(y_trim, sr))
    # Classical
    feats.update(classical_features(path))
    # Nonlinear
    feats.update(nonlinear_features(path))
    # Wavelet
    feats.update(wavelet_band_energy(y_trim))
    # Scattering
    feats.update(wavelet_scattering_feats(y_trim))
    # Embeddings stats
    if TRANSFORMERS_AVAILABLE:
        try:
            emb_stats = extract_wav2vec_stats(y_trim, sr)
            feats.update(emb_stats)
        except Exception as e:
            print("Embedding extraction failed for", path, ":", e)
    # metadata
    feats["_file"] = path
    # session date: try parse from text file next to wav (if exists) else file mtime
    txt_path = os.path.splitext(path)[0] + ".txt"
    if os.path.exists(txt_path):
        try:
            with open(txt_path, 'r', encoding='utf-8') as fh:
                txt = fh.read().strip()
            feats["_transcript"] = txt
            # Optionally parse date from filename if encoded - here we fallback to mtime
        except:
            feats["_transcript"] = ""
    else:
        feats["_transcript"] = ""
    feats["_date"] = datetime.fromtimestamp(os.path.getmtime(path))
    return feats

def analyze_patient(patient_group, patient_subfolder=None):
    """Analyze a single top-level patient group (e.g., 'emma') OR a specific subfolder inside it"""
    if patient_subfolder:
        root_dir = os.path.join(BASE_DIR, patient_group, patient_subfolder)
    else:
        root_dir = os.path.join(BASE_DIR, patient_group)
    wavs = sorted(glob.glob(os.path.join(root_dir, "**", "*.wav"), recursive=True))
    if len(wavs) == 0:
        print("No wavs found for", root_dir); return None
    print(f"Found {len(wavs)} wav files for analysis under {root_dir}")
    rows = []
    for w in tqdm(wavs):
        try:
            r = extract_features_for_file(w)
            rows.append(r)
        except Exception as e:
            print("Error extracting", w, e)
    df = pd.DataFrame(rows)
    # Sort by date if available
    df = df.sort_values("_date").reset_index(drop=True)
    # Baseline normalization: baseline = mean of first N sessions (N=1 or 2)
    baseline_N = 2 if len(df) >=2 else 1
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    baseline = df.loc[:baseline_N-1, numeric_cols].mean(skipna=True)
    # compute normalized columns and slope per feature (slope per day)
    slope_map = {}
    for col in numeric_cols:
        base = baseline.get(col, np.nan)
        df[f"{col}_norm"] = (df[col] - base) / (abs(base) + 1e-9) if not np.isnan(base) else np.nan
        # slope over time (value vs days)
        if df.shape[0] >= 2:
            x = np.array([(d - df["_date"].iloc[0]).days for d in df["_date"]]).reshape(-1,1)
            yv = df[col].fillna(method='ffill').fillna(0).values.reshape(-1,1)
            lr = LinearRegression().fit(x, yv)
            slope_map[col] = float(lr.coef_[0][0])
        else:
            slope_map[col] = np.nan
    # create a simple weighted deterioration_score using a few clinically important features
    # weight signs chosen so that higher deterioration_score -> worse
    weights = {
        "spl_mean_db": -1.0,   # decreasing SPL (negative slope) is worse so negative weight
        "jitter_local": 1.0,
        "shimmer_local": 1.0,
        "rpde_approx": 1.0
    }
    # compute weighted sum of slopes
    weighted_sum = 0.0
    for feat, w in weights.items():
        s = slope_map.get(feat, 0.0)
        if np.isnan(s): s = 0.0
        weighted_sum += w * s
    # attach deterioration score per session as cumulative slope proxy (we show the aggregate slope as single line)
    # For plotting over sessions, we compute per-session deterioration_score as weighted sum of normalized features
    def per_session_score(row):
        s = 0.0
        for feat, w in weights.items():
            ncol = f"{feat}_norm"
            if ncol in row and not np.isnan(row[ncol]):
                s += w * row[ncol]
        return s
    df["deterioration_score"] = df.apply(per_session_score, axis=1)
    # CUSUM on deterioration_score
    def simple_cusum(series, threshold=2.0, drift=0.0):
        pos = 0.0; neg = 0.0; alarms = []
        mu = np.nanmean(series); sigma = np.nanstd(series) + 1e-9
        for x in series:
            s = (x - mu) / sigma if sigma>0 else 0.0
            pos = max(0, pos + s - drift)
            neg = min(0, neg + s + drift)
            if pos > threshold:
                alarms.append("pos_alarm")
            elif abs(neg) > threshold:
                alarms.append("neg_alarm")
            else:
                alarms.append("ok")
        return alarms
    df["cusum_flag"] = simple_cusum(df["deterioration_score"].fillna(0).values)
    # Plot deterioration line (per-session score) with baseline ribbon
    plt.figure(figsize=(10,4))
    dates = df["_date"]
    scores = df["deterioration_score"]
    plt.plot(dates, scores, marker='o', label="deterioration_score")
    # baseline ribbon: +- 2*std of baseline sessions (if available)
    baseline_vals = df.loc[:baseline_N-1, "deterioration_score"]
    if not baseline_vals.empty:
        bmean = baseline_vals.mean(); bstd = baseline_vals.std()
        plt.fill_between(dates, bmean - 2*bstd, bmean + 2*bstd, color='gray', alpha=0.2, label="baseline ± 2σ")
    plt.title(f"Deterioration score over time: {patient_group} / {patient_subfolder if patient_subfolder else ''}")
    plt.ylabel("Deterioration score (higher = worse)")
    plt.xticks(rotation=25)
    plt.grid(True); plt.legend()
    plt.show()
    # Print summary
    print("Feature slopes (per day) sample (subset):")
    sample_slopes = {k: slope_map[k] for k in list(slope_map.keys())[:10]}
    print(sample_slopes)
    print("CUSUM flags (last 10 sessions):", df["cusum_flag"].tail(10).tolist())
    print("Cell 10 finished: patient analysis plotted and summary printed.")
    return df, slope_map

# Example usage for patient 'emma' top-level group (analyze entire 'emma' group)
if "emma" in os.listdir(BASE_DIR):
    df_emma, slopes_emma = analyze_patient("emma")
else:
    print("Patient group 'emma' not found under BASE_DIR.")
